In [ ]:
#this is an implement of LeNet-5 convolutional neural network with input image size 28 × 28
#you can find more details on https://pub.towardsai.net/the-architecture-implementation-of-lenet-5-eef03a68d1f7

In [29]:
import torch as th
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = th.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = th.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

print("Number of training samples:", len(train_dataset))
print("Number of testing samples:", len(test_dataset))

Number of training samples: 60000
Number of testing samples: 10000


In [20]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

epochs = 20
learning_rate = 0.01
momentum = 0.9

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2,)

        self.fc1 = nn.Linear(16 * 4 * 4, 120) # Since the input image size is 28*28, after two convolutions and poolings, the image size may be smaller than 5*5, so the input size here is 16*4*4
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = self.avgpool(x)
        x = F.tanh(self.conv2(x))
        x = self.avgpool(x)
        
        x = x.view(-1, 16 * 4 * 4)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x


In [34]:
def train(model, train_loader, optimizer, criterion):
    model.train()

    for epoch in range(epochs):
        out_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            out_loss += loss.item()

            if i % 300 == 299:  
                print("Epoch: %d, Batch: %3d, Loss: %.3f" % (epoch + 1, i + 1, out_loss / 300))
                out_loss = 0.0
    
    print('Finished Training')
    return model

def test(model, test_loader):
    correct = 0
    total = 0

    with th.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = th.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %0.2f %%' % (100 * correct / total))

In [4]:
def main():
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
    model = LeNet5().to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    criterion = nn.CrossEntropyLoss()

    lenet5 = train(model, train_loader, optimizer, criterion)
    test(lenet5, test_loader)
    

In [35]:
if __name__ == "__main__":
    main()

Epoch: 1, Batch: 300, Loss: 2.285
Epoch: 1, Batch: 600, Loss: 1.970
Epoch: 1, Batch: 900, Loss: 1.673
Epoch: 2, Batch: 300, Loss: 1.597
Epoch: 2, Batch: 600, Loss: 1.567
Epoch: 2, Batch: 900, Loss: 1.554
Epoch: 3, Batch: 300, Loss: 1.542
Epoch: 3, Batch: 600, Loss: 1.535
Epoch: 3, Batch: 900, Loss: 1.525
Epoch: 4, Batch: 300, Loss: 1.521
Epoch: 4, Batch: 600, Loss: 1.515
Epoch: 4, Batch: 900, Loss: 1.510
Epoch: 5, Batch: 300, Loss: 1.506
Epoch: 5, Batch: 600, Loss: 1.506
Epoch: 5, Batch: 900, Loss: 1.501
Epoch: 6, Batch: 300, Loss: 1.499
Epoch: 6, Batch: 600, Loss: 1.496
Epoch: 6, Batch: 900, Loss: 1.497
Epoch: 7, Batch: 300, Loss: 1.493
Epoch: 7, Batch: 600, Loss: 1.493
Epoch: 7, Batch: 900, Loss: 1.491
Epoch: 8, Batch: 300, Loss: 1.489
Epoch: 8, Batch: 600, Loss: 1.490
Epoch: 8, Batch: 900, Loss: 1.488
Epoch: 9, Batch: 300, Loss: 1.487
Epoch: 9, Batch: 600, Loss: 1.484
Epoch: 9, Batch: 900, Loss: 1.488
Epoch: 10, Batch: 300, Loss: 1.483
Epoch: 10, Batch: 600, Loss: 1.484
Epoch: 10, B